In [1]:
import os, sys, json, glob
import numpy as np
import matplotlib.pyplot as plt
import girder_client
import histomicstk.utils as htk_utils
from cStringIO import StringIO
from IPython.display import Image as IPImage  ## PIL has an Image Function as well, so want to keep these distinct
import histomicstk.utils as htk_utils
import io, random
from PIL import Image, ImageDraw
import logging

logging.getLogger("requests").setLevel(logging.WARNING)

%matplotlib inline

INFO:girder:Created LRU Cache for 'tilesource' with 104 maximum size
INFO:root:Notice: Could not import MapnikTileSource


In [2]:
gc = girder_client.GirderClient(apiUrl="http://candygram.neurology.emory.edu:8080/api/v1")
gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [3]:
## Connect to girder and get the cohort ID's for all the TCGA collections
tcgaCohorts = gc.get('/tcga/cohort')  ## This gets me the folderID for all the TCGA cohorts
cohortInfo = dict([(x['name'],x['_id']) for x in tcgaCohorts['data']])

In [19]:
slidesInCohort = gc.get('/tcga/cohort/%s/images?limit=%d' % (cohortInfo['gbm'],50))
## PROBABLY should write a generator to grab all the slides instead of just specifying some giant number

In [33]:
sampleSlide = slidesInCohort['data'][47]
sampleSlide

{u'_id': u'597ddca192ca9a000de46fac',
 u'baseParentId': u'5956b84792ca9a000dd670c8',
 u'baseParentType': u'collection',
 u'created': u'2017-07-30T13:18:25.510000+00:00',
 u'creatorId': u'5956b83b92ca9a000d9c0488',
 u'description': u'',
 u'folderId': u'597e22bd92ca9a000de595f4',
 u'largeImage': {u'fileId': u'597ddca192ca9a000de46fad',
  u'sourceName': u'svs'},
 u'lowerName': u'tcga-02-0010-01z-00-dx2.5334831b-8e1f-4b61-bbf6-0f6e950a1b2f.svs',
 u'name': u'TCGA-02-0010-01Z-00-DX2.5334831b-8e1f-4b61-bbf6-0f6e950a1b2f.svs',
 u'size': 837094511,
 u'tcga': {u'barcode': u'TCGA-02-0010-01Z-00-DX2',
  u'case': u'TCGA-02-0010',
  u'caseId': u'597e22bd92ca9a000de595f0',
  u'cohort': u'gbm',
  u'label': u'TCGA-02-0010',
  u'type': u'image',
  u'uuid': u'5334831b-8e1f-4b61-bbf6-0f6e950a1b2f'},
 u'updated': u'2017-07-30T18:36:17.695000+00:00'}

In [34]:
sID = sampleSlide['_id']
lowResMag = 1.25
lowResImg = gc.get('/item/%s/tiles/region?magnification=%s' % ( sID, lowResMag),jsonResp=False)

In [14]:
# IPImage(lowResImg.content)

In [35]:
lowResPILimage = Image.open(io.BytesIO(lowResImg.content))
im_fgnd_mask_lres = htk_utils.simple_mask(np.asarray(lowResPILimage))

In [ ]:
# plt.rcParams['image.cmap'] = 'gray'
# plt.imshow(im_fgnd_mask_lres)

In [36]:
### NEXT STEP--- I need to grab every POINT in the MASK.. and then grab 100 Random Tiles...
## FIGuRE OUT THE INDEXES OF ALL POINTS IN THE MASK
(YmaskPts,XmaskPts)  = np.nonzero(im_fgnd_mask_lres)  ## This returns a  Tuples of 2 arrays  X and Y coordinates of non zero points

## Zip Y and X into Coords
maskCoords = zip(YmaskPts,XmaskPts)
### So now grab some random points from this array

In [37]:
slideBaseName = sampleSlide['name'].split(".")[0]
print slideBaseName
out_path = "/data/train/gbm/"

TCGA-02-0010-01Z-00-DX2


In [38]:
# To save the tiles into file
tilename =[]

random.shuffle(maskCoords)  ### shuffle the points and then chose however many pts I wnat to grab tiles for
outputRes = 20
scaleFactor = outputRes/ lowResMag  ### Need to multiply the Y And X coords by this number to get the target coordinates

maxx = len(maskCoords)
tilecount=0
for idx, c in enumerate(maskCoords):
    print("Image %s of %s" % (idx, maxx))
    curTile = gc.get('/item/%s/tiles/region?magnification=%s&top=%d&left=%d&regionWidth=%d&regionHeight=%d' 
                       % ( sID, outputRes, c[0]*scaleFactor, c[1]*scaleFactor,256,256),jsonResp=False)
    img = Image.open(io.BytesIO(curTile.content))
    avg = np.average(img)
    if avg > 150 and avg < 210:
#         plt.figure()
#         plt.imshow(img)
        tilename = slideBaseName + '_%dx_%d_%d.png' % (outputRes, c[0], c[1]) 
        img.save(out_path + tilename)
        tilecount+=1
        
    if tilecount ==50000:
        break
#         plt.savefig(out_path + tilename)  

Image 0 of 4282322
Image 1 of 4282322
Image 2 of 4282322
Image 3 of 4282322
Image 4 of 4282322
Image 5 of 4282322
Image 6 of 4282322
Image 7 of 4282322
Image 8 of 4282322
Image 9 of 4282322
Image 10 of 4282322
Image 11 of 4282322
Image 12 of 4282322
Image 13 of 4282322
Image 14 of 4282322
Image 15 of 4282322
Image 16 of 4282322
Image 17 of 4282322
Image 18 of 4282322
Image 19 of 4282322
Image 20 of 4282322
Image 21 of 4282322
Image 22 of 4282322
Image 23 of 4282322
Image 24 of 4282322
Image 25 of 4282322
Image 26 of 4282322
Image 27 of 4282322
Image 28 of 4282322
Image 29 of 4282322
Image 30 of 4282322
Image 31 of 4282322
Image 32 of 4282322
Image 33 of 4282322
Image 34 of 4282322
Image 35 of 4282322
Image 36 of 4282322
Image 37 of 4282322
Image 38 of 4282322
Image 39 of 4282322
Image 40 of 4282322
Image 41 of 4282322
Image 42 of 4282322
Image 43 of 4282322
Image 44 of 4282322
Image 45 of 4282322
Image 46 of 4282322
Image 47 of 4282322
Image 48 of 4282322
Image 49 of 4282322
Image 50 o

Image 396 of 4282322
Image 397 of 4282322
Image 398 of 4282322
Image 399 of 4282322
Image 400 of 4282322
Image 401 of 4282322
Image 402 of 4282322
Image 403 of 4282322
Image 404 of 4282322
Image 405 of 4282322
Image 406 of 4282322
Image 407 of 4282322
Image 408 of 4282322
Image 409 of 4282322
Image 410 of 4282322
Image 411 of 4282322
Image 412 of 4282322
Image 413 of 4282322
Image 414 of 4282322
Image 415 of 4282322
Image 416 of 4282322
Image 417 of 4282322
Image 418 of 4282322
Image 419 of 4282322
Image 420 of 4282322
Image 421 of 4282322
Image 422 of 4282322
Image 423 of 4282322
Image 424 of 4282322
Image 425 of 4282322
Image 426 of 4282322
Image 427 of 4282322
Image 428 of 4282322
Image 429 of 4282322
Image 430 of 4282322
Image 431 of 4282322
Image 432 of 4282322
Image 433 of 4282322
Image 434 of 4282322
Image 435 of 4282322
Image 436 of 4282322
Image 437 of 4282322
Image 438 of 4282322
Image 439 of 4282322
Image 440 of 4282322
Image 441 of 4282322
Image 442 of 4282322
Image 443 of 

Image 787 of 4282322
Image 788 of 4282322
Image 789 of 4282322
Image 790 of 4282322
Image 791 of 4282322
Image 792 of 4282322
Image 793 of 4282322
Image 794 of 4282322
Image 795 of 4282322
Image 796 of 4282322
Image 797 of 4282322
Image 798 of 4282322
Image 799 of 4282322
Image 800 of 4282322
Image 801 of 4282322
Image 802 of 4282322
Image 803 of 4282322
Image 804 of 4282322
Image 805 of 4282322
Image 806 of 4282322
Image 807 of 4282322
Image 808 of 4282322
Image 809 of 4282322
Image 810 of 4282322
Image 811 of 4282322
Image 812 of 4282322
Image 813 of 4282322
Image 814 of 4282322
Image 815 of 4282322
Image 816 of 4282322
Image 817 of 4282322
Image 818 of 4282322
Image 819 of 4282322
Image 820 of 4282322
Image 821 of 4282322
Image 822 of 4282322
Image 823 of 4282322
Image 824 of 4282322
Image 825 of 4282322
Image 826 of 4282322
Image 827 of 4282322
Image 828 of 4282322
Image 829 of 4282322
Image 830 of 4282322
Image 831 of 4282322
Image 832 of 4282322
Image 833 of 4282322
Image 834 of 

Image 1170 of 4282322
Image 1171 of 4282322
Image 1172 of 4282322
Image 1173 of 4282322
Image 1174 of 4282322
Image 1175 of 4282322
Image 1176 of 4282322
Image 1177 of 4282322
Image 1178 of 4282322
Image 1179 of 4282322
Image 1180 of 4282322
Image 1181 of 4282322
Image 1182 of 4282322
Image 1183 of 4282322
Image 1184 of 4282322
Image 1185 of 4282322
Image 1186 of 4282322
Image 1187 of 4282322
Image 1188 of 4282322
Image 1189 of 4282322
Image 1190 of 4282322
Image 1191 of 4282322
Image 1192 of 4282322
Image 1193 of 4282322
Image 1194 of 4282322
Image 1195 of 4282322
Image 1196 of 4282322
Image 1197 of 4282322
Image 1198 of 4282322
Image 1199 of 4282322
Image 1200 of 4282322
Image 1201 of 4282322
Image 1202 of 4282322
Image 1203 of 4282322
Image 1204 of 4282322
Image 1205 of 4282322
Image 1206 of 4282322
Image 1207 of 4282322
Image 1208 of 4282322
Image 1209 of 4282322
Image 1210 of 4282322
Image 1211 of 4282322
Image 1212 of 4282322
Image 1213 of 4282322
Image 1214 of 4282322
Image 1215

Image 1544 of 4282322
Image 1545 of 4282322
Image 1546 of 4282322
Image 1547 of 4282322
Image 1548 of 4282322
Image 1549 of 4282322
Image 1550 of 4282322
Image 1551 of 4282322
Image 1552 of 4282322
Image 1553 of 4282322
Image 1554 of 4282322
Image 1555 of 4282322
Image 1556 of 4282322
Image 1557 of 4282322
Image 1558 of 4282322
Image 1559 of 4282322
Image 1560 of 4282322
Image 1561 of 4282322
Image 1562 of 4282322
Image 1563 of 4282322
Image 1564 of 4282322
Image 1565 of 4282322
Image 1566 of 4282322
Image 1567 of 4282322
Image 1568 of 4282322
Image 1569 of 4282322
Image 1570 of 4282322
Image 1571 of 4282322
Image 1572 of 4282322
Image 1573 of 4282322
Image 1574 of 4282322
Image 1575 of 4282322
Image 1576 of 4282322
Image 1577 of 4282322
Image 1578 of 4282322
Image 1579 of 4282322
Image 1580 of 4282322
Image 1581 of 4282322
Image 1582 of 4282322
Image 1583 of 4282322
Image 1584 of 4282322
Image 1585 of 4282322
Image 1586 of 4282322
Image 1587 of 4282322
Image 1588 of 4282322
Image 1589

...

Image 50622 of 4282322
Image 50623 of 4282322
Image 50624 of 4282322
Image 50625 of 4282322
Image 50626 of 4282322
Image 50627 of 4282322
Image 50628 of 4282322
Image 50629 of 4282322
Image 50630 of 4282322
Image 50631 of 4282322
Image 50632 of 4282322
Image 50633 of 4282322
Image 50634 of 4282322
Image 50635 of 4282322
Image 50636 of 4282322
Image 50637 of 4282322
Image 50638 of 4282322
Image 50639 of 4282322
Image 50640 of 4282322
Image 50641 of 4282322
Image 50642 of 4282322
Image 50643 of 4282322
Image 50644 of 4282322
Image 50645 of 4282322
Image 50646 of 4282322
Image 50647 of 4282322
Image 50648 of 4282322
Image 50649 of 4282322
Image 50650 of 4282322
Image 50651 of 4282322
Image 50652 of 4282322
Image 50653 of 4282322
Image 50654 of 4282322
Image 50655 of 4282322
Image 50656 of 4282322
Image 50657 of 4282322
Image 50658 of 4282322
Image 50659 of 4282322
Image 50660 of 4282322
Image 50661 of 4282322
Image 50662 of 4282322
Image 50663 of 4282322
Image 50664 of 4282322
Image 50665

Image 50979 of 4282322
Image 50980 of 4282322
Image 50981 of 4282322
Image 50982 of 4282322
Image 50983 of 4282322
Image 50984 of 4282322
Image 50985 of 4282322
Image 50986 of 4282322
Image 50987 of 4282322
Image 50988 of 4282322
Image 50989 of 4282322
Image 50990 of 4282322
Image 50991 of 4282322
Image 50992 of 4282322
Image 50993 of 4282322
Image 50994 of 4282322
Image 50995 of 4282322
Image 50996 of 4282322
Image 50997 of 4282322
Image 50998 of 4282322
Image 50999 of 4282322
Image 51000 of 4282322
Image 51001 of 4282322
Image 51002 of 4282322
Image 51003 of 4282322
Image 51004 of 4282322
Image 51005 of 4282322
Image 51006 of 4282322
Image 51007 of 4282322
Image 51008 of 4282322
Image 51009 of 4282322
Image 51010 of 4282322
Image 51011 of 4282322
Image 51012 of 4282322
Image 51013 of 4282322
Image 51014 of 4282322
Image 51015 of 4282322
Image 51016 of 4282322
Image 51017 of 4282322
Image 51018 of 4282322
Image 51019 of 4282322
Image 51020 of 4282322
Image 51021 of 4282322
Image 51022

Image 51338 of 4282322
Image 51339 of 4282322
Image 51340 of 4282322
Image 51341 of 4282322
Image 51342 of 4282322
Image 51343 of 4282322
Image 51344 of 4282322
Image 51345 of 4282322
Image 51346 of 4282322
Image 51347 of 4282322
Image 51348 of 4282322
Image 51349 of 4282322
Image 51350 of 4282322
Image 51351 of 4282322
Image 51352 of 4282322
Image 51353 of 4282322
Image 51354 of 4282322
Image 51355 of 4282322
Image 51356 of 4282322
Image 51357 of 4282322
Image 51358 of 4282322
Image 51359 of 4282322
Image 51360 of 4282322
Image 51361 of 4282322
Image 51362 of 4282322
Image 51363 of 4282322
Image 51364 of 4282322
Image 51365 of 4282322
Image 51366 of 4282322
Image 51367 of 4282322
Image 51368 of 4282322
Image 51369 of 4282322
Image 51370 of 4282322
Image 51371 of 4282322
Image 51372 of 4282322
Image 51373 of 4282322
Image 51374 of 4282322
Image 51375 of 4282322
Image 51376 of 4282322
Image 51377 of 4282322
Image 51378 of 4282322
Image 51379 of 4282322
Image 51380 of 4282322
Image 51381

Image 51698 of 4282322
Image 51699 of 4282322
Image 51700 of 4282322
Image 51701 of 4282322
Image 51702 of 4282322
Image 51703 of 4282322
Image 51704 of 4282322
Image 51705 of 4282322
Image 51706 of 4282322
Image 51707 of 4282322
Image 51708 of 4282322
Image 51709 of 4282322
Image 51710 of 4282322
Image 51711 of 4282322
Image 51712 of 4282322
Image 51713 of 4282322
Image 51714 of 4282322
Image 51715 of 4282322
Image 51716 of 4282322
Image 51717 of 4282322
Image 51718 of 4282322
Image 51719 of 4282322
Image 51720 of 4282322
Image 51721 of 4282322
Image 51722 of 4282322
Image 51723 of 4282322
Image 51724 of 4282322
Image 51725 of 4282322
Image 51726 of 4282322
Image 51727 of 4282322
Image 51728 of 4282322
Image 51729 of 4282322
Image 51730 of 4282322
Image 51731 of 4282322
Image 51732 of 4282322
Image 51733 of 4282322
Image 51734 of 4282322
Image 51735 of 4282322
Image 51736 of 4282322
Image 51737 of 4282322
Image 51738 of 4282322
Image 51739 of 4282322
Image 51740 of 4282322
Image 51741

Image 52055 of 4282322
Image 52056 of 4282322
Image 52057 of 4282322
Image 52058 of 4282322
Image 52059 of 4282322
Image 52060 of 4282322
Image 52061 of 4282322
Image 52062 of 4282322
Image 52063 of 4282322
Image 52064 of 4282322
Image 52065 of 4282322
Image 52066 of 4282322
Image 52067 of 4282322
Image 52068 of 4282322
Image 52069 of 4282322
Image 52070 of 4282322
Image 52071 of 4282322
Image 52072 of 4282322
Image 52073 of 4282322
Image 52074 of 4282322
Image 52075 of 4282322
Image 52076 of 4282322
Image 52077 of 4282322
Image 52078 of 4282322
Image 52079 of 4282322
Image 52080 of 4282322
Image 52081 of 4282322
Image 52082 of 4282322
Image 52083 of 4282322
Image 52084 of 4282322
Image 52085 of 4282322
Image 52086 of 4282322
Image 52087 of 4282322
Image 52088 of 4282322
Image 52089 of 4282322
Image 52090 of 4282322
Image 52091 of 4282322
Image 52092 of 4282322
Image 52093 of 4282322
Image 52094 of 4282322
Image 52095 of 4282322
Image 52096 of 4282322
Image 52097 of 4282322
Image 52098

In [ ]:
type(img)

In [ ]:
polygon = []
for i,c in enumerate(maskCoords):
    polygon.append((c[1],c[0]))
print(len(polygon))

In [ ]:
height, width = np.asarray(lowResPILimage).shape[:-1]

img = Image.new('L', (width, height), 0)
ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
mask = np.array(img)

plt.imshow(mask)

In [ ]:
import logging
random.shuffle(maskCoords)  ### shuffle the points and then chose however many pts I wnat to grab tiles for
logging.getLogger('requests').setLevel(logging.CRITICAL)

outputRes = 20
scaleFactor = outputRes/ lowResMag  ### Need to multiply the Y And X coords by this number to get the target coordinates
fig=plt.figure(figsize=(15, 10))

### Data is YX

columns = 10
rows = 10
idx = 1

#for c in mask[:100]:
#    print c[0], c[1]
for c in maskCoords[:100]:
    curTile = gc.get('/item/%s/tiles/region?magnification=%s&top=%d&left=%d&regionWidth=%d&regionHeight=%d' 
                       % ( sID, outputRes, c[0]*scaleFactor, c[1]*scaleFactor,256,256),jsonResp=False)
    img = Image.open(io.BytesIO(curTile.content))
    avg = np.average(img)
    if avg > 150 and avg < 210:
        fig.add_subplot(rows,columns,idx)
        plt.imshow(img)
        idx+=1
plt.show(fig)

In [ ]:
mask

In [ ]:
maskCoords[0]

In [ ]:

from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')


In [ ]:
im_fgnd_hl = im_fgnd_mask_lres.copy()

for c in maskCoords[:25]:
    print(c[0],c[1])
    im_fgnd_hl[c[0],c[1]] = 5

plt.imshow(im_fgnd_hl)

# # .shape

# im_fgnd_hl.max()


#print(im_fgnd_hl.max())

In [ ]:
im_fgnd_hl[im_fgnd_hl == 2].sum()

In [ ]:
#IPImage(curTile.content)

plt.imshow( Image.open(io.IOycurTile.content)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

w=10
h=10
fig=plt.figure(figsize=(8, 8))
columns = 4
rows = 5
for i in range(1, columns*rows +1):
    img = np.random.randint(10, size=(h,w))
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

In [ ]:
# # # import base64, io
# # # from PIL import Image
# # # import cv2
# # # # Take in base64 string and return cv image
# # # def stringToRGB(base64_string):
# # #     imgdata = base64.b64decode(str(base64_string))
# # #     image = Image.open(io.BytesIO(imgdata))
# # #     return cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)

# from PIL import Image as PILImage
# # import StringIO

# # # img = PILImage.open

# RawImageToPIL =  PILImage.open(StringIO(thumbImage.content)) ### This is painful.. 
# # # RawImageToPIL.save(oj(macroImagePath,macroName))

# im_fgnd_mask_lres = htk_utils.simple_mask(PILImage)
# # ## Remember the image from the girder response is a string, it needs to become an imaimgdata)
# # #im_lres would be the RGB image of the entire slide at low-res (1.25 x) that you can get from the get region REST end point